# Environment setup
In this notebook, you will quickly set up the Databricks environment for use with Azure Data Lake Storage Gen2 (ADLS Gen2).

### Attach notebook to you cluster
Before executing any cells in your notebook, you need to attach it to your cluster. In the notebook's toolbar, select the drop down arrow next to Detached, and then select your cluster under attach to.

Run the next cell to initialize your filesystem and mount it with DBFS.

## Connect to Azure Machine Learning Workspace

In [3]:
import azureml.core

from azureml.core import Workspace, Dataset
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.datastore import Datastore

service_principal_id = dbutils.secrets.get(keyVaultScope, "databricks-aml-demo-sp-client-id")
service_principal_password = dbutils.secrets.get(keyVaultScope, "databricks-aml-demo-sp-client-key")
tenant_id = dbutils.secrets.get(keyVaultScope, "azure-tenant-id")


# PRODUCTION AML WORKSPACE
workspace_name = "amls-databricks"
subscription_id = dbutils.secrets.get(keyVaultScope, "azure-subscription-id")
resource_group = "jp-databricks"


svc_pr = ServicePrincipalAuthentication(
    service_principal_id=service_principal_id,
    service_principal_password=service_principal_password,
    tenant_id=tenant_id
    )

ws = Workspace(workspace_name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resource_group,
               auth=svc_pr)

print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace amls-databricks at location centralus

## Mount Azure Blob Storage, copy sample dataset and register in Azure Machine Learning

In [5]:
# Get default datastore and upload dataset
datastore = ws.get_default_datastore()

# Variable declarations. These will be accessible by any calling notebook.
keyVaultScope = "databricks-aml-demo"
blobaccountname = dbutils.secrets.get(keyVaultScope, "blob-accountname")

# Mount the Azure Blob Storage.
if not any(mount.mountPoint == f"/mnt/{datastore.container_name}" for mount in dbutils.fs.mounts()):
  dbutils.fs.mount(
    source = f"wasbs://{datastore.container_name}@{blobaccountname}.blob.core.windows.net", 
    mount_point = f"/mnt/{datastore.container_name}",
    extra_configs = {f"fs.azure.account.key.{blobaccountname}.blob.core.windows.net":dbutils.secrets.get(keyVaultScope, key = "blob-account-key")})

In [6]:
# Copy data from dbfs to temp storage for upload and registration in AML
dbutils.fs.cp ('dbfs:/databricks-datasets/samples/population-vs-price/data_geo.csv', f'/mnt/{datastore.container_name}/population-vs-price/data_geo.csv')

Out[35]: True

In [7]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, '/population-vs-price/data_geo.csv')], infer_column_types=True)
dataset.register(workspace=ws, name='data_geo', description='data_geo training data')

Out[36]: {
 "source": [
 "('workspaceblobstore', 'population-vs-price/data_geo.csv')"
 ],
 "definition": [
 "GetDatastoreFiles",
 "ParseDelimited",
 "DropColumns",
 "SetColumnTypes"
 ],
 "registration": {
 "id": "1d357c4d-bc8d-4fdc-a996-bdd06f81d49a",
 "name": "data_geo",
 "version": 1,
 "description": "data_geo training data",
 "workspace": "Workspace.create(name='amls-databricks', subscription_id='[REDACTED]', resource_group='jp-databricks')"
 }
}

## Next

You have completed the environment setup.